In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.optimize import curve_fit
import csv

#plt.style.use(['science', 'notebook', 'grid', 'dark_background', 'bright'])
plt.style.use(['science', 'notebook', 'grid', 'default', 'bright'])    #used to get white plot backgrounds

SMALL_SIZE = 25
MEDIUM_SIZE = 35
BIGGER_SIZE = 45
#SMALL_SIZE = 35
#MEDIUM_SIZE = 45
#BIGGER_SIZE = 55

plot_height=12    #Plot size parameters
grat=1.618

plt.rcParams['figure.figsize'] = (plot_height*grat, plot_height)    #Default plot size

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

plt.rcParams['axes.grid'] = False

In [3]:
def get_scan(number):    #returns the data file scan number in Spice format
    const = 3
    if (number > 999):    #no leading zeros
        zeros = 0
    else:
        if (number == 0):
            zeros = const
        else:
            zeros = const - int(np.floor(np.log10(number)))
    return '0'*zeros + str(number)

default_order = ['detector','monitor','time','time-normed counts','nut_pre','nut_post','b1','b2','fgen_ampl','fgen_freq', \
                  'fgen_units','fgen_offset','fgen_func','fgen_term','tslit','rot2','anatran','m1','m2','mtilt']
default_dict = {i[1]:i[0] for i in enumerate(default_order)}

def load_data(path, names, dest, write_data=False, overwrite=False, reorder=False):    #data loading function from Spice format
    scan_nums,dates,times,scan_titles,parms,data =[[],[],[],[],[],[]]
    for i in range(len(names)):
        file = open(path + names[i])
        lines = file.readlines()
        lines = [i.replace('\n','') for i in lines]    #removes new line characters
        file.close()
        
        scan_nums.append(lines[0][2::])    #line number determined by Spice format
        scan_nums[i] = scan_nums[i].split(' = ',)[1]
        dates.append(lines[1][2::])
        dates[i] = dates[i].split(' = ',)[1]
        times.append(lines[2][2::])
        times[i] = times[i].split(' = ',)[1]
        scan_titles.append(lines[10][2::])
        scan_titles[i] = scan_titles[i].split(' = ',)[1]
        parms.append(lines[29][7::].split())
        parms[i].append('time-normed counts')
        data.append([j.split()[1::] for j in lines[30::] if (j[0] != '#')])    #assigns the data
        data[i] = [[float(data[i][j][k]) for j in range(len(data[i]))] for k in range(len(data[i][0]))]    #converts to float
        data[i] = [list(j) for j in zip(*data[i])]    #transposes table
        
        time_normed_counts = [data[i][j][2]/data[i][j][1] if (data[i][j][1] != 0) else 0 for j in range(len((data[i])))]
        for j in range(len(data[i])):
            data[i][j].append(time_normed_counts[j])
        
        if ((reorder == True) and (parms[i] != default_order)):    #rearranges the columns to default_order
            swap_i = []    #columns to swap
            for j in range(len(parms[i])):
                swap_i.append([j, default_dict[parms[i][j]]])

            swaped_data = [ [0]*len(data[i][0]) for _ in range(len(data[i]))]    #matrix to hold swaped values
            for j in range(len(data[i])):    #slow swapping method, may need some work...
                for k in range(len(data[i][j])):
                    for m in range(len(swap_i)):
                        if (swap_i[m][1] == k):
                            swaped_data[j][k] = data[i][j][m]
                            parms[i][m] = default_order[swap_i[m][0]]    
            data[i] = swaped_data
            
        if (write_data == True):    #writes data to csv files
            if (overwrite == True):
                c_str = 'w'
            else:
                c_str = 'x'    #Fails if the file already exists
            windows_fname = scan_titles[i]    #Cleans the Spice file names for Windows
            invalid = '<>:"/\|?*'
            for char in invalid:
                windows_fname = windows_fname.replace(char,'')
            with open(dest_path + "scan" + str(scan_nums[i]) + "_" + windows_fname + ".csv", c_str, newline='') as csv_file:
                writer = csv.writer(csv_file)
                writer.writerow([scan_titles[i]])
                writer.writerow([dates[i]])
                writer.writerow([times[i]])
                writer.writerow(parms[i])
                for j in range(len(data[i])):
                    writer.writerow(data[i][j])
                    
    #returns the list [scan number, scan date, scan time start, user input scan titles, matrix of data from scan]
    return [scan_nums,dates,times,scan_titles,parms,data]

data_path = "C:/Users/samck/Dropbox/CG4B RF coil experiment/exp8/Datafiles/"
dest_path = "C:/Users/samck/Desktop/HFIR_RF/"    #WARNING: may overwrite files in destination folder!

#file_range = [1,8,52,110]
file_range = [i for i in range(1,516)]
file_list = ["CG-4B_exp0008_scan" + get_scan(i) + ".dat" for i in file_range]

Num,Date,Time,Title,Parms,Data = load_data(data_path, file_list, dest_path, write_data=True, overwrite=False, reorder=True)


FileExistsError: [Errno 17] File exists: 'C:/Users/samck/Desktop/HFIR_RF/scan1_Detector Scan.csv'

In [14]:
Parms[0]

['detector',
 'monitor',
 'time',
 'time-normed counts',
 'nut_pre',
 'nut_post',
 'b1',
 'b2',
 'fgen_ampl',
 'fgen_freq',
 'fgen_units',
 'fgen_offset',
 'fgen_func',
 'fgen_term',
 'tslit',
 'rot2',
 'anatran',
 'm1',
 'm2',
 'mtilt']